In [ ]:
import torch
import os
import numpy as np
import pandas as pd
import torch_geometric
from torch.nn import Linear, Module, BatchNorm1d, ReLU, Dropout, MSELoss, ModuleList, LSTM, L1Loss
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch.optim import Adadelta

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Importing Data

In [ ]:
directory = '/data'

files = os.listdir(directory)

data = []

for file in files:
    # Add all values from files into list
    data.append(pd.read_csv(file).values[:, -1])
    
# input list as data for torch model
flows = torch.from_numpy(np.stack(flows)).float()

# Creating Graph Structure

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2, 2, 3, 0, 3, 0, 4, 3, 4],
                           [1, 0, 2, 1, 3, 2, 3, 0, 4, 0, 4, 3]], dtype=torch.long)
x = torch.tensor([[1], [1], [1], [1], [1], [1], [1], [1], [1]], dtype=torch.float)

data = Data(x=x.to(device), edge_index=edge_index.to(device), flows=flows.to(device))

# Model Implementation

### Setup is fairly similar to all other Torch neural networks
#### Uses Adadelta algorithm for optimization

In [ ]:
def train(st, data, time, loss_fn, optimizer):
    st.train()

    optimizer.zero_grad()

    output = st(data, time)

    loss = loss_fn(output, data.flows[:, time])

    loss.backward()

    optimizer.step()

    return loss.item()

In [ ]:
@torch.no_grad()
def eval(st, data, times, returned=None):
    st.eval()

    errors = []
    outputs = []
    for time in times:
        output = st(data, time)
        if returned:
            outputs.append(output)
        error = L1Loss()(output, data.flows[:, time])
        errors.append(error)
        # print(output, data.flows[:, time])
    if returned:
        return outputs
    return torch.tensor(errors).mean()

In [ ]:
class SpatioTemporal(Module):
    def __init__(self, input_dim, gnn_dims, node_pooling_dims, graph_pooling_dims, rnn_dim, ff_dims, gnn_dropout_rate, ff_dropout_rate, rnn_dropout_rate):
        super(SpatioTemporal, self).__init__()
        self.input_dim = input_dim
        self.gnn_dims = gnn_dims
        self.node_pooling_dims = node_pooling_dims
        self.graph_pooling_dims = graph_pooling_dims
        self.rnn_dim = rnn_dim
        self.ff_dims = ff_dims
        self.gnn_dropout_rate = gnn_dropout_rate
        self.ff_dropout_rate = ff_dropout_rate
        self.rnn_dropout_rate = rnn_dropout_rate

        ## Implementing GNN Layers
        dims = [self.input_dim] + gnn_dims
        self.gnn_bn_layers = ModuleList(
            [
               BatchNorm1d(dim) for dim in dims[1:]
            ]
        )
        
        self.gnn_layers = ModuleList(
            [
               GCNConv(dim, dims[i+1]) for i, dim in enumerate(dims[:-1])
            ]
        )

        ## Node Pooling
        dims = [self.gnn_dims[-1]] + self.node_pooling_dims
        self.node_pooling_layers = ModuleList(
            [
               Linear(dim, dims[i+1]) for i, dim in enumerate(dims[:-1])
            ]
        )

        self.node_pooling_bn_layers = ModuleList(
            [
               BatchNorm1d(dim) for dim in dims[1:]
            ]
        )

        ## Graph Pooling
        dims = [self.node_pooling_dims[-1]] + self.graph_pooling_dims
        self.graph_pooling_layers = ModuleList(
            [
               Linear(dim, dims[i+1]) for i, dim in enumerate(dims[:-1])
            ]
        )

        ## Adding RNN Layers 
        self.rnn = LSTM(self.graph_pooling_dims[-1] + self.ff_dims[-1], self.rnn_dim, 2, dropout=self.rnn_dropout_rate)


        dims = [self.rnn_dim] + self.ff_dims
        self.ff_layers = ModuleList(
            [
               Linear(dim, dims[i+1]) for i, dim in enumerate(dims[:-1])
            ]
        )

        self.activation = ReLU()
        self.gnn_dropout = Dropout(self.gnn_dropout_rate)
        self.ff_dropout = Dropout(self.ff_dropout_rate)

    def forward(self, data, timestep):
        x = data.x

        # GNN
        for conv, bn in zip(self.gnn_layers, self.gnn_bn_layers):
            
            x = conv(x, edge_index=data.edge_index)
            x = bn(x)
            x = self.activation(x)
            x = self.gnn_dropout(x)
        
        # Node Pooling 
        for ff, bn in zip(self.node_pooling_layers, self.node_pooling_bn_layers):
            x = ff(x)
            x = bn(x)
            x = self.activation(x)
            x = self.ff_dropout(x) 
        
        x = x.sum(0).reshape(1, -1)

        # Graph Pooling 
        for ff in self.graph_pooling_layers:
            x = ff(x)
            # x = bn(x)
            x = self.activation(x)
            x = self.ff_dropout(x) 

        # RNN
        x = x.repeat(timestep + 1, 1)

        global device
        x_time = torch.hstack((torch.zeros(flows.shape[0]).reshape(-1, 1).to(device), data.flows[:, :timestep])).t()

        x = torch.hstack((x_time, x))
        
        x, (h, c) = self.rnn(x)
        
        # Regression
        x = x[-1, :]
        for ff in self.ff_layers[:-1]:
            x = ff(x)
            # x = bn(x)
            x = self.activation(x)
            x = self.ff_dropout(x)
        
        x = self.ff_layers[-1](x)
        
        return self.ff_dropout(x).reshape(-1)

# Training Function

In [ ]:
def pipeline_for_one_config(st, data, loss_fn, optimizer):
    train_errors = []
    val_errors = []
    test_errors = []
    train_errors.append(eval(st, data, range(1, 750)))
    val_errors.append(eval(st, data, range(751, 975)))
    test_errors.append(eval(st, data, range(976, 1200)))

    test_outputs = []
    for epoch in range(500):
        train_errors.append(eval(st, data, range(1, 750)))
        val_errors.append(eval(st, data, range(751, 975)))
        test_errors.append(eval(st, data, range(976, 1200)))
        test_outputs.append(eval(st, data, range(976, 1200), True))

        epoch_loss = 0
        for time in range(1, 80):
            epoch_loss += train(st, data, time, loss_fn, optimizer)
    train_errors.append(eval(st, data, range(1, 750)))
    val_errors.append(eval(st, data, range(751, 975)))
    test_errors.append(eval(st, data, range(976, 1200)))

    i = torch.tensor(val_errors).argmin()
    return val_errors[i], test_errors[i], i, test_outputs[i]

# Run Algorithm

In [ ]:
# Adjust hyperparameters here:

gnn_dim = 64
np_dim = 80
gp_dim = 80
rnn_dim = 384
ff_dim = 300
dropout = 0.25
lr = 1e-4
st = SpatioTemporal(1, [gnn_dim, gnn_dim], [np_dim, np_dim], [gp_dim, gp_dim], rnn_dim, [ff_dim, 8], dropout, dropout, dropout).to(device)
optimizer = Adam(st.parameters(), lr=lr)
loss_fn = MSELoss()
dev_error, test_error, i, predictions = pipeline_for_one_config(st, data, loss_fn, optimizer)

# Create Weighted Graph Network Structure

In [ ]:
weighted_values = torch.stack(predictions)[0, :]
edges = [(0, 1), (1, 2), (2, 3), (0, 3), (0, 4), (3, 4)]
graph = np.zeros(25).reshape(-1, 5)
for i in range(5):
    for j in range(5):
        for key, prediction in zip(edges, weighted_values):
            if i in key and j in key and i != j:
                graph[i][j] = prediction
                break
                
# graph now contains the outputted weighted graph network structure